In [1]:
from tensorscaling import scale, unit_tensor, random_tensor, marginal
import numpy as np

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [2]:
shape = [3, 3, 3]
targets = [(.5, .25, .25), (.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

Result(success=True, iterations=72, max_dist=9.608957572434757e-05, ..., log_cap=-0.25742089309561533)

We can also access the scaling matrices and the final scaled state:

In [3]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

[[ 1.50384058+0.j          0.        +0.j          0.        +0.j        ]
 [-0.19778787-0.26030572j  0.59775617+0.j          0.        +0.j        ]
 [ 0.66888475-0.02754442j  0.37908721-0.2345403j   0.98118214+0.j        ]] 

[[ 1.34433751e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j        ]
 [-3.96672521e-01+0.02073528j  6.88112011e-01+0.j
   0.00000000e+00+0.j        ]
 [ 9.39780147e-04+0.47189597j -2.17646459e-01-0.10794224j
   1.08130290e+00+0.j        ]] 

[[ 1.52424954+0.j          0.        +0.j          0.        +0.j        ]
 [-0.14205244+0.0645477j   0.70835218+0.j          0.        +0.j        ]
 [-0.04876855-0.07106133j  0.01169964+0.05841099j  0.50179658+0.j        ]]


Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [4]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = .5
targets = [(.5, .5)] * 4

scale(W, targets, eps=1e-4, max_iterations=1000)

Result(success=False, iterations=1000, max_dist=0.027330403424211745, ..., log_cap=None)

To see more clearly what is going on, we can set the `verbose` flag:

In [5]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

scaling tensor of shape (2, 2, 2, 2) and type float64
target spectra:
  0: (0.5, 0.5)
  1: (0.5, 0.5)
  2: (0.5, 0.5)
  3: (0.5, 0.5)
#000: max_dist = 0.35355339 @ sys = 2
#001: max_dist = 0.47140452 @ sys = 1
#002: max_dist = 0.56568542 @ sys = 0
#003: max_dist = 0.62853936 @ sys = 3
#004: max_dist = 0.58232323 @ sys = 2
#005: max_dist = 0.59305730 @ sys = 1
#006: max_dist = 0.59545834 @ sys = 0
#007: max_dist = 0.59262283 @ sys = 3
#008: max_dist = 0.59353004 @ sys = 2
#009: max_dist = 0.59357133 @ sys = 1
#010: max_dist = 0.59340661 @ sys = 0
did not converge!


We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [6]:
shape = [3, 3, 3]
targets = [(.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Result(success=True, iterations=31, max_dist=8.84476038809367e-07, ..., log_cap=-0.4033679502754076)

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [7]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

[[0.3534 +0.j      0.01552+0.01882j 0.01224-0.00617j]
 [0.01552-0.01882j 0.32809-0.j      0.00661-0.01156j]
 [0.01224+0.00617j 0.00661+0.01156j 0.31851-0.j     ]] 

[[ 0.4+0.j -0. -0.j -0. -0.j]
 [-0. +0.j  0.3-0.j -0. +0.j]
 [-0. +0.j -0. -0.j  0.3-0.j]] 

[[ 0.7-0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0.2+0.j -0. +0.j]
 [ 0. -0.j -0. -0.j  0.1+0.j]]
